In [1]:
import polars as pl

import rustystats as rs

data = pl.read_parquet("https://raw.githubusercontent.com/PricingFrontier/pricing-data-example/917c853e256df8d5814721ab56f72889a908bb08/data/processed/frequency_set.parquet")

In [2]:
#creates analysis/exploration.json

exploration = rs.explore_data(
    data=data,
    response="ClaimCount",
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ],
    exposure="Exposure",
)

In [3]:
# fits model

model = rs.glm(
    formula="ClaimCount ~ VehPower + VehAge + C(Area) + C(Region)",
    data=data,
    family="negbinomial",
    offset="Exposure"
).fit()

In [4]:
#writes analysis/diagnostics
model.diagnostics(
    data=data,
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ]
)

ModelDiagnostics(model_summary={'formula': 'ClaimCount ~ VehPower + VehAge + C(Area) + C(Region)', 'family': 'NegativeBinomial(theta=1.3802)', 'link': 'log', 'n_observations': 678012, 'n_parameters': 28, 'degrees_of_freedom_residual': 677984, 'converged': True, 'iterations': 6}, convergence_details=ConvergenceDetails(max_iterations_allowed=25, iterations_used=6, converged=True, reason='converged'), fit_statistics={'deviance': 196741.36192725928, 'null_deviance': 192800.37653563166, 'deviance_explained': -0.020440755679225875, 'log_likelihood': -98370.68096362964, 'aic': 196797.36192725928, 'bic': 197117.31569470395, 'pearson_chi2': 1678923.192319597, 'dispersion_deviance': 0.2901858479363219, 'dispersion_pearson': 2.4763463331282107}, loss_metrics={'family_deviance_loss': 0.2901740260019395, 'mse': 0.055126770295653854, 'mae': 0.09871081359506938, 'rmse': 0.2347909076085653}, calibration={'ae_ratio': 0.991, 'hl_pvalue': 0.0, 'problem_deciles': [{'decile': 0, 'ae': 6.44, 'n': 67124, 'ae